In [3]:
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U accelerate jupyter ipywidgets
!pip install gymnasium
import gymnasium as gym
import math
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    GenerationConfig,
    pipeline,
)
import torch
from transformers import pipeline


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 8.6 MB/s eta 0:00:00


In [4]:
models_dict = {
    "StableLM Zephyr 3B": "stabilityai/stable-zephyr-3b-dpo"
}

model_id = models_dict["StableLM Zephyr 3B"]

In [5]:
tokenizer = AutoTokenizer.from_pretrained(model_id, device=0)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/5.21k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/587 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [6]:
# Running this may take a minute
pipe = pipeline("text-generation", model=model_id, device_map="auto", tokenizer=tokenizer, torch_dtype=torch.bfloat16)

config.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/5.59G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [80]:
job_description = """You are a large language model playing a game called CartPole. You can only answer 'Left' or 'Right'"""

In [99]:
zero_shot_prompt = """In which direction the cart of a cart pole should be moved if the pole direction is #degrees degrees
off to the #direction?"""

one_shot_prompt = """When the pole tilts to the right the cart should be moved to the right.
In which direction the cart should be moved if the pole direction is #degrees degrees off to the #direction?"""

few_shot_prompt = """When the pole tilts to the right the cart should be moved to the right.
When the pole tilts to the left the cart should be moved to the left.
In which direction the cart should be moved if the pole direction is #degrees degrees off to the #direction"""

Chain_of_Thought_prompt = """Q: In which direction the cart of a cart pole should be moved if
the pole direction is 5 degrees off to the left
A: Left
Q: In which direction the cart of a cart pole should be moved if
the pole direction is 5 degrees off to the right
A: Right
Q: In which direction the cart of a cart pole should be moved if
the pole direction is #degrees degrees off to the #direction?"""


prompts = [zero_shot_prompt,one_shot_prompt,few_shot_prompt,Chain_of_Thought_prompt]
#prompts = [Chain_of_Thought_prompt]
prompts = {
    "zero_shot_prompt": zero_shot_prompt,
    "one_shot_prompt": one_shot_prompt,
    "few_shot_prompt": few_shot_prompt,
    "Chain_of_Thought_prompt": Chain_of_Thought_prompt
}

In [100]:
temperatures = [0.01, 0.3, 0.6, 0.9]

In [101]:
def generate(question, temperature=0.7, max_new_tokens=512):
    messages = [
        {
            "role": "system",
            "content": job_description,
        },
    ]
    messages.append({"role": "user", "content": question})
    prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    outputs = pipe(prompt, max_new_tokens=max_new_tokens, do_sample=True, temperature=temperature, top_k=50, top_p=0.95)
    output = outputs[0]["generated_text"]
    messages.append({"role": "assistant", "content": output})
    #print(output)
    response_start = output.rfind('<|assistant|>')
    text_output = output[response_start + len('<|assistant|>'):]
    return text_output

In [102]:
def last_word_used(text):
    # Find the last index of "left" and "right"
    left_index = text.rfind("left")
    right_index = text.rfind("right")

    # Compare the indices to determine the last word used
    if left_index > right_index:
        return "left"
    elif right_index > left_index:
        return "right"
    else:
        # Both words have the same index, handle this case as needed
        return "Both words appear at the same position"

In [103]:
def insertValuesInPrompt(prompt, degrees, direction):
    prompt = prompt.replace("#degrees", str(degrees))
    prompt = prompt.replace("#direction", direction)
    #print(prompt)
    return prompt

In [115]:
def runExperiment(episodes,promptKey, temperature):
    env = gym.make('CartPole-v1')#, render_mode="human")
    prompt = prompts[promptKey]
    for i_episode in range(episodes):
        observation = env.reset()
        rad = observation[0][2]
        rewards = 0
        randomActions = 0
        leftActions = 0
        rightActions = 0

        for t in range(250):
            env.render()

            #action = env.action_space.sample()

            degrees = round(math.degrees(rad), 2)
            direction = "right"
            if degrees < 0:
                direction = "left"
                degrees = degrees * -1

            output = generate(insertValuesInPrompt(prompt, degrees, direction), temperature, 10)

            action = 0
            #print("OUTPUT")
            #print(output)
            #print("ENDOUTPUT")
            if last_word_used(output.lower()) == "right":
                action = 1
                actionText = 'right'
                rightActions+=1
            elif last_word_used(output.lower()) == "left":
                action = 0
                actionText = 'left'
                leftActions+=1
            else:
                actionText = 'random'
                randomActions+=1
                env.action_space.sample() #Inconclusive answer, random action
            #print(output)
            #print(str(degrees) + " " + direction + " answer: "+actionText+" action: " + str(action))
            observation, reward, terminated, truncated, info = env.step(action)
            rad = observation[2]
            rewards += reward

            if terminated:
                print('prompt: {}, temperature:{}, timesteps: {}, total rewards: {}, left: {}, right: {}, random: {}'.format(promptKey, temperature, t+1, rewards,leftActions,rightActions,randomActions))
                break

    env.close()


In [113]:
def runExperiments(episodes,prompts, temperatures):
  for promptKey in prompts.keys():
    for temperature in temperatures:
      runExperiment(episodes, promptKey, temperature)

In [116]:
runExperiments(1,prompts, temperatures)

prompt: zero_shot_prompt, temperature:0.01, timesteps: 9, total rewards: 9.0, left: 0, right: 0, random: 9
prompt: zero_shot_prompt, temperature:0.3, timesteps: 9, total rewards: 9.0, left: 0, right: 0, random: 9
prompt: zero_shot_prompt, temperature:0.6, timesteps: 10, total rewards: 10.0, left: 0, right: 0, random: 10
prompt: zero_shot_prompt, temperature:0.9, timesteps: 10, total rewards: 10.0, left: 0, right: 1, random: 9
prompt: one_shot_prompt, temperature:0.01, timesteps: 10, total rewards: 10.0, left: 0, right: 0, random: 10
prompt: one_shot_prompt, temperature:0.3, timesteps: 9, total rewards: 9.0, left: 0, right: 0, random: 9
prompt: one_shot_prompt, temperature:0.6, timesteps: 9, total rewards: 9.0, left: 0, right: 0, random: 9
prompt: one_shot_prompt, temperature:0.9, timesteps: 11, total rewards: 11.0, left: 1, right: 1, random: 9
prompt: few_shot_prompt, temperature:0.01, timesteps: 9, total rewards: 9.0, left: 0, right: 0, random: 9
prompt: few_shot_prompt, temperature:0